In [10]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [3]:
players_dataset = pd.read_csv('../datasets/international_played_foreign_players.csv')
players_dataset.head()

,player_id,player_name,country,played_international,bowling_style,batting_style,player_role
0,2,BB McCullum,New Zealand,2002 - 2016,"Right arm Medium, Right arm Offbreak",Right hand Bat,Wicketkeeper Batter
1,3,RT Ponting,Australia,1995 - 2012,Right arm Medium,Right hand Bat,Top order Batter
2,4,DJ Hussey,Australia,2008 - 2013,Right arm Offbreak,Right hand Bat,Allrounder
3,5,Mohammad Hafeez,Pakistan,2003 - 2021,Right arm Offbreak,Right hand Bat,Allrounder
4,9,JH Kallis,South Africa,1995 - 2014,Right arm Fast medium,Right hand Bat,Allrounder


In [12]:
player_search_page_url = "https://stats.espncricinfo.com/ci/engine/stats/index.html"

In [25]:
test_df = players_dataset[players_dataset["player_id"]<5]
test_df.head()

,player_id,player_name,country,played_international,bowling_style,batting_style,player_role
0,2,BB McCullum,New Zealand,2002 - 2016,"Right arm Medium, Right arm Offbreak",Right hand Bat,Wicketkeeper Batter
1,3,RT Ponting,Australia,1995 - 2012,Right arm Medium,Right hand Bat,Top order Batter
2,4,DJ Hussey,Australia,2008 - 2013,Right arm Offbreak,Right hand Bat,Allrounder


In [32]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

all_player_dfs = []

for index,row in players_dataset.iterrows():
    player_name = row["player_name"]
    player_id = row["player_id"]
    driver.get(player_search_page_url)
    print(player_name)
    
    # Locate the search input field and type the player name
    search_box = driver.find_element(By.NAME, "search")
    search_box.send_keys(player_name)
    
    # Locate and click the submit button
    submit_button = driver.find_element(By.NAME, "submit_analysis")
    submit_button.click()
    
#     time.sleep(1)
    
    players_tab = driver.find_element(By.ID, "player")
    players_tab.click()
    
#     time.sleep(1)
    try:
        t20i_link = driver.find_element(By.LINK_TEXT, "Twenty20 Internationals player")

        t20i_link.click()
        players_dataset.at[index, 'played_t20i'] = 'Yes'

        # T20I Query Page
        match_by_match_radio_button = driver.find_element(By.XPATH, '//input[@type="radio" and @name="view" and @value="match"]')
        match_by_match_radio_button.click()

        submit_button = driver.find_element(By.NAME, "submit_results")
        submit_button.click()

        # Getting table data

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        tables = soup.find_all('table', {'class': 'engineTable'})
        for table in tables:
            caption = table.find('caption')
            if caption and caption.text.strip() == "Match by match list":
                headers = [th.text.strip() for th in table.find('thead').find_all('th')]
                rows = []
                for row in table.find('tbody').find_all('tr'):
                    cells = [td.text.strip() for td in row.find_all('td')]

                    # Extract the match link from the column with title="view the scorecard for this row"
                    td_link = row.find('td', {'style': 'white-space: nowrap;'})
                    match_link = None  # Default to None if no link is found

                    if td_link:
                        link = td_link.find('a', {'title': 'view the scorecard for this row'})
                        if link:
                            match_link = link.get('href')  # Extract the link (relative)

                    # Add the match_link to the row
                    cells.append(match_link)

                    rows.append(cells)

                # Add match_link as a new column
                headers.append('match_link')  # Add 'match_link' to headers
                df = pd.DataFrame(rows, columns=headers)
                df['player_id'] = player_id  # Add player_id column
                all_player_dfs.append(df)
    except Exception as e:
        players_dataset.at[index, 'played_t20i'] = 'No'

final_df = pd.concat(all_player_dfs, ignore_index=True)    
final_df.to_csv('../datasets/international_played_foreign_players_matches_details.csv',index=False)
print("successful")

BB McCullum
RT Ponting
DJ Hussey
Mohammad Hafeez
JH Kallis
CL White
MV Boucher
AA Noffke
ML Hayden
MEK Hussey
JDP Oram
JR Hopes
KC Sangakkara
SM Katich
SR Watson
DS Lehmann
SK Warne
L Ronchi
ST Jayasuriya
SM Pollock
S Chanderpaul
LRPL Taylor
AC Gilchrist
A Symonds
SB Styris
WPUJC Vaas
DPMD Jayawardene
S Sohal
B Lee
Kamran Akmal
Shahid Afridi
DJ Bravo
GC Smith
RR Sarwan
Shoaib Malik
MF Maharoof
DW Steyn
HH Gibbs
DNT Zoysa
SE Marsh
Sohail Tanvir
Salman Butt
BJ Hodge
Umar Gul
SP Fleming
JA Morkel
AB de Villiers
LPC Silva
Misbah-ul-Haq
Mohammad Asif
GD McGrath
M Ntini
CK Kapugedera
T Taibu
TM Dilshan
AD Mascarenhas
LA Pomersbach
Younis Khan
Shoaib Akhtar
Abdur Razzak
DR Smith
CRD Fernando
JP Duminy
A Flintoff
T Thushara
JD Ryder
KP Pietersen
T Henderson
RS Bopara
CH Gayle
MC Henriques
FH Edwards
RJ Quiney
BAW Mendis
DL Vettori
MN van Wyk
RE van der Merwe
SM Harwood
DA Warner
RJ Harris
M Morkel
AD Mathews
J Botha
SL Malinga
Mashrafe Mortaza
GJ Bailey
AB McDonald
Mohammad Ashraful
OA Shah
R 

In [33]:
players_dataset.to_csv('../datasets/international_played_foreign_players_updated.csv',index=False)